In [ ]:
import pandas as pd
import csv
from collections import Counter

# CSV dosyasını oku
df = pd.read_csv("/content/hepsi_birlestirilmis3.csv", on_bad_lines='skip')

# Etiketleri içeren sütun adını kontrol edin ve uygun şekilde düzenleyin
# Örneğin 'duygu_label' sütunundaki etiketleri analiz edeceğiz
labels_column = 'duygu_label'

# Etiketleri saklayacağımız bir liste oluştur
all_labels = []

# JSON nesnesinden etiketleri çıkar
for _, row in df.iterrows():
    # Convert the value to a string if it's not NaN
    if not pd.isna(row[labels_column]):
        try:
            entities = json.loads(str(row[labels_column]))
            for entity in entities:
                if isinstance(entity, dict) and 'labels' in entity:
                    all_labels.extend(entity['labels'])
        except json.JSONDecodeError:
            print(f"JSON decode error for row: {row[labels_column]}")

# Etiketlerin sayısını hesapla
label_counts = Counter(all_labels)

# Pozitif, negatif ve nötr etiketlerin sayısını al
positive_count = label_counts.get("positive", 0)
negative_count = label_counts.get("negative", 0)
neutral_count = label_counts.get("neutral", 0)

print(f"Positive count: {positive_count}")
print(f"Negative count: {negative_count}")
print(f"Neutral count: {neutral_count}")


NameError: name 'json' is not defined

In [ ]:
!pip install nltk spacy pandas simpletransformers scikit-learn transformers jpype1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.3 MB/s eta 0:00:00
  

In [ ]:
import nltk
import spacy
import pandas as pd
import json
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from simpletransformers.ner import NERModel, NERArgs
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from transformers import AutoModelForTokenClassification, AutoTokenizer, TrainingArguments, Trainer
import torch
from datasets import Dataset, load_metric
import jpype
from jpype import JClass, getDefaultJVMPath, startJVM
import csv

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import json

# CSV dosyasını oku
df = pd.read_csv("/content/hepsi_birlestirilmis3.csv", on_bad_lines='skip')

# Yalnızca 'text' ve 'label' sütunlarını kullan
df = df[['text', 'label']]

# JSON formatında veriyi hazırlamak
data = {
    "Text": df['text'].tolist(),
    "Entities": []
}

for index, row in df.iterrows():
    entity_list = []

    # 'label' sütunundaki etiketleri ekle
    if not pd.isna(row['label']):
        try:
            label_entities = json.loads(row['label'])
            for entity in label_entities:
                entity_list.append({
                    "start": entity["start"],
                    "end": entity["end"],
                    "text": entity["text"],
                    "labels": entity["labels"]
                })
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError at row {index}: {e}")

    data["Entities"].append(entity_list)

# JSON dosyasını kaydet
with open("/content/output_son.json", "w", encoding="utf-8") as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

print("JSON dosyası başarıyla oluşturuldu.")


JSONDecodeError at row 114: Expecting value: line 1 column 1 (char 0)
JSON dosyası başarıyla oluşturuldu.


In [ ]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
import json

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')

# JSON dosyasının yolunu belirtin
file_path = r"/content/output_son.json"

# JSON dosyasını aç ve içeriği bir değişkene ata
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Load the dataset
df = pd.DataFrame({
    "Text": data["Text"],
    "Entities": data["Entities"]
})

# Function to tokenize text and annotate entities
def tokenize_and_annotate(df):
    tokens = []
    labels = []
    for _, row in df.iterrows():
        tweet = row['Text']
        entities = row['Entities']

        # Ensure tweet is a string
        if not isinstance(tweet, str):
            tweet = str(tweet)

        tokenized_text = nltk.word_tokenize(tweet)
        token_labels = ['O'] * len(tokenized_text)

        for entity in entities:
            start_idx = entity['start']
            end_idx = entity['end']
            entity_text = entity['labels'][0]  # Assuming there is always one label

            # Find token indices for entity
            char_idx = 0
            token_start = -1
            token_end = -1
            for i, token in enumerate(tokenized_text):
                char_idx = tweet.find(token, char_idx)
                if char_idx == start_idx:
                    token_start = i
                if char_idx + len(token) >= end_idx:
                    token_end = i
                    break
                char_idx += len(token)

            if token_start != -1 and token_end != -1:
                token_labels[token_start] = f'B-{entity_text}'
                for i in range(token_start + 1, token_end + 1):
                    token_labels[i] = f'I-{entity_text}'

        tokens.append(tokenized_text)
        labels.append(token_labels)

    return tokens, labels

tokens, labels = tokenize_and_annotate(df)

# Creating a DataFrame with tokens and labels
annotated_df = pd.DataFrame({
    'Tokens': tokens,
    'Labels': labels
})

# Saving to Excel
annotated_df.to_excel('/content/data.xlsx', index=False)

print("Tokenized and annotated dataset has been saved to annotated_dataset(4).xlsx")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenized and annotated dataset has been saved to annotated_dataset(4).xlsx


In [ ]:
!pip install simpletransformers

In [ ]:

import pandas as pd
from simpletransformers.ner import NERModel, NERArgs
import logging
from sklearn.model_selection import train_test_split

# 1. Veriyi Yükle ve Hazırla
df = pd.read_excel('/content/data.xlsx')

# String liste temsillerini gerçek listelere dönüştür
df['Tokens'] = df['Tokens'].apply(eval)
df['Labels'] = df['Labels'].apply(eval)

# SimpleTransformers için gerekli formata getir
data = []
sentence_id = 0
for tokens, labels in zip(df['Tokens'], df['Labels']):
    for token, label in zip(tokens, labels):
        data.append([sentence_id, token, label])
    sentence_id += 1

# DataFrame'i oluştur
data_df = pd.DataFrame(data, columns=['sentence_id', 'words', 'labels'])

# Eğitim ve test veri setlerine ayır
train_df, eval_df = train_test_split(data_df, test_size=0.1)

# Etiketlerin listesini çıkar
labels_list = list(set([label for labels in df['Labels'] for label in labels]))

# 2. Modeli Konfigüre Et ve Eğit

# Logging yapılandırması
logging.basicConfig(level=logging.INFO)  # INFO seviyesi loglama

model_args = NERArgs()
model_args.num_train_epochs = 10  # Epoch sayısını azalt
model_args.train_batch_size = 32  # Batch boyutunu küçült
model_args.eval_batch_size = 32
model_args.learning_rate = 3e-5  # Learning rate ayarı
model_args.save_steps = -1  # Adım başına kaydetmeyi devre dışı bırak
model_args.save_model_every_epoch = False  # Her epoch sonunda kaydetmeyi devre dışı bırak
model_args.overwrite_output_dir = True  # Çıktı dizinini üzerine yazmak için eklenen argüman
model_args.labels_list = labels_list  # Etiketleri model argümanlarına ekle
model_args.fp16 = True  # FP16 kullanarak GPU hızını artır

# Modeli oluştur
model = NERModel(
    "bert",
    "dbmdz/bert-base-turkish-128k-uncased",
    args=model_args,
    use_cuda=True
)

# Modeli eğit
model.train_model(train_df, eval_data=eval_df)

# 3. Modeli Değerlendir
result, model_outputs, preds_list = model.eval_model(eval_df)
print(result)

# 4. Yeni Cümleler Üzerinde Tahmin Yap
predictions, raw_outputs = model.predict(["kickturkey platformunda superonline hızlı ve kesintisiz yayın deneyimi sunuyor"])
print(predictions)

prediction2, raw_outputs = model.predict(["ben hayatımda bu kadar rezil bir şirket görmedim turkcell türk telekom"])
print(prediction2)

prediction3, raw_outputs = model.predict(["rezilsin turkcell bu ne kepazelik böyle türk telekom dan bile kötüsün"])
print(prediction3)

prediction4, raw_outputs = model.predict(["twitch de yayın izlerken bir anda internet kesildi diğer yayın platformlarında bunu yaşamıyorum twitch ile ilgili bir problemin mi var superonline"])
print(prediction4)

prediction5, raw_outputs = model.predict(["ajet de 3 saatlik rötar varmış bu ne kardeşim oyuncak mıyız biz pegasus da bu sorunu hiç yaşamamıştım"])
print(prediction5)

prediction5, raw_outputs = model.predict(["dünkü thy uçuşu muazzamdı buradan hizmeti geçen herkese teşekkürler türk hava yolları"])
print(prediction5)




Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/110 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/110 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'ajet': 'O'}, {'de': 'O'}, {'3': 'O'}, {'saatlik': 'O'}, {'rötar': 'O'}, {'varmış': 'O'}, {'bu': 'O'}, {'ne': 'O'}, {'kardeşim': 'O'}, {'oyuncak': 'O'}, {'mıyız': 'O'}, {'biz': 'O'}, {'pegasus': 'B-Entity'}, {'da': 'O'}, {'bu': 'O'}, {'sorunu': 'O'}, {'hiç': 'O'}, {'yaşamamıştım': 'O'}]]


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'dünkü': 'I-Entity'}, {'thy': 'B-Entity'}, {'uçuşu': 'O'}, {'muazzamdı': 'O'}, {'buradan': 'O'}, {'hizmeti': 'O'}, {'geçen': 'O'}, {'herkese': 'O'}, {'teşekkürler': 'O'}, {'türk': 'B-Entity'}, {'hava': 'I-Entity'}, {'yolları': 'I-Entity'}]]


In [ ]:
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score

# CSV dosyasını oku
df = pd.read_csv("hepsi_birlestirilmis3.csv")

# Veriyi uygun formata getir
data = []
for _, row in df.iterrows():
    # Initialize entities as an empty list
    entities = []

    # Convert the value to a string if it's not NaN
    if not pd.isna(row['duygu_label']):
        # Load the JSON object
        try:
            entities = json.loads(str(row['duygu_label']))
        except json.JSONDecodeError:
            print(f"JSON decode error for row: {row['duygu_label']}")
            continue

    # Iterate over the entities and append to the data list
    for entity in entities:
        if isinstance(entity, dict) and 'text' in entity and 'labels' in entity:
            data.append({
                "text": entity["text"],
                "labels": entity["labels"][0]  # Assuming there is always one label
            })
        else:
            print(f"Skipping invalid entity: {entity}")

# Eğitim verisini DataFrame'e dönüştür
train_df = pd.DataFrame(data)

# Veriyi eğitim ve doğrulama setlerine böl
train_data, eval_data = train_test_split(train_df, test_size=0.1, random_state=42)

# Etiketlerin listesini çıkar
labels_list = ["negative", "neutral", "positive"]
labels_map = {"negative": 0, "neutral": 1, "positive": 2}

# Model argümanlarını belirle
args = ClassificationArgs()
args.num_train_epochs = 10  # Eğitim süresini artıracağız
args.learning_rate = 3e-5
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32
args.save_steps = -1  # Adım başına kaydetmeyi devre dışı bırak
args.save_model_every_epoch = False  # Her epoch sonunda kaydetmeyi devre dışı bırak
args.labels_list = labels_list  # Etiketleri model argümanlarına ekle
args.output_dir = "sentiment_model_output"  # Kaydedileceği konum adı
args.labels_map = None  # labels_map'i None olarak ayarla

# BERT tabanlı duygu analizi modelini oluştur
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-cased', args=args, num_labels=len(labels_list))

# Modeli eğit
model.train_model(train_data, eval_df=eval_data)

# Modeli kaydet
model.save_model("sentiment_model_output")

# Metrik fonksiyonlarını tanımla
def compute_metrics(preds, labels):
    assert len(preds) == len(labels)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    return {
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
    }

# Doğrulama sonuçlarını al ve metrikleri hesapla
result, model_outputs, wrong_predictions = model.eval_model(eval_data, compute_metrics=compute_metrics)

print("Validation Results:", result)

JSON decode error for row: duygu_label


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/158 [00:00<?, ?it/s]

...

Running Epoch 10 of 10:   0%|          | 0/158 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/18 [00:00<?, ?it/s]

Validation Results: {'mcc': 0.31587320356626347, 'compute_metrics': {'precision': 0.8871307060644489, 'recall': 0.7119856887298748, 'f1_score': 0.7702643200201614}, 'eval_loss': 0.7446513142850664}


In [ ]:
from simpletransformers.ner import NERModel
from simpletransformers.classification import ClassificationModel

# NER modelini yükle
ner_model = NERModel("bert", "/content/outputs", use_cuda=True)

# Duygu analizi modelini yükle
sentiment_model = ClassificationModel("bert", "/content/sentiment_model_output", use_cuda=True)

# Örnek cümle
sentence = "turkcell kullanışlı çok memnunum"

# Cümle üzerinde NER tahmini yap
ner_predictions, _ = ner_model.predict([sentence])

# NER çıktısından varlıkları ve etiketlerini çıkar
entities = []
current_entity = []
current_label = None

for token_info in ner_predictions[0]:
    for token, label in token_info.items():
        if label.startswith('B-') or (label.startswith('I-') and current_label != label[2:]):
            if current_entity:
                entities.append((" ".join(current_entity), current_label))
                current_entity = []
            current_entity.append(token)
            current_label = label[2:]
        elif label.startswith('I-') and current_label == label[2:]:
            current_entity.append(token)
        else:
            if current_entity:
                entities.append((" ".join(current_entity), current_label))
                current_entity = []
            current_label = None

if current_entity:
    entities.append((" ".join(current_entity), current_label))
print(sentence)
# Varlıkların yazdırılması
print("Entities:")
for entity_text, label in entities:
    print(f"Entity: {entity_text}, Label: {label}")

# Varlıklar üzerinde duygu analizi yap
entity_texts = [entity_text for entity_text, _ in entities]
sentiment_predictions, _ = sentiment_model.predict(entity_texts)

# Sentiment predictions formatını kontrol et
print("Sentiment Predictions Format:")
print(sentiment_predictions)

# Sentiment analizi sonuçlarını yazdır
print("Sentiment Analysis:")
sentiment_labels = ["negative", "neutral", "positive"]
for entity_text, sentiment in zip(entity_texts, sentiment_predictions):
    print(f"Entity: {entity_text}, Sentiment: {sentiment}")

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

turkcell kullanışlı çok memnunum
Entities:
Entity: turkcell, Label: Entity


0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Sentiment Predictions Format:
['negative']
Sentiment Analysis:
Entity: turkcell, Sentiment: negative
